In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import os

list_email = ''
email_customer = ''

# Variaveis
db_pbi = ''
filepath = ''
filepathteste = ''

filepath_consultor = ''

path_to_read = ''
file_to_read_old = ''

url_xpto = ''

# Historico
hist_path = ''
arch_name = ''
dt_arch = datetime.now().strftime("%Y%m%d_%H%M")

In [2]:
read_file_old = os.path.join(path_to_read, file_to_read_old)
df_old = pd.read_excel(rf'{read_file_old}')
df_old['DF_NAME'] = ''

In [3]:
df_consultor = pd.read_csv(rf'{filepath_consultor}', sep=';', encoding='latin-1')

In [4]:
def extract_xpto(url):
    # URL of the API (replace with actual URL if different)

    # Fazer a requisição GET
    response = requests.get(url, verify=False)

    # Verificar o status da requisição
    if response.status_code == 200:
        # Requisição bem-sucedida
        html_content = response.text
    else:
        # Erro na requisição
        print(f"Erro: {response.status_code}")
        exit()

    # Criar um objeto BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Encontrar a tabela
    table = soup.find("table")

    # Extrair linhas e colunas da tabela
    rows = table.find_all("tr")
    columns = [th.text for th in rows[0].find_all("th")]

    # Extrair dados das células da tabela
    data = []
    for row in rows[1:]:
        data.append([td.text for td in row.find_all("td")])

    # Criar o DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df

In [5]:
df_new = extract_xpto(url_xpto)
df_new['DF_NAME'] = ''

C:\Users\rafael.ferrari\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kbi.desk.ms'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [6]:
def rename_columns(df, dict_df):
    df = df.rename(columns=dict_df)
    return df


def fit_df(df, list_columns):
    colcom = list(set(list_columns).intersection(set(df.columns)))
    coldif = list(set(list_columns) - set(df.columns))
    df = df[colcom]
    for each_element in coldif:
        df[each_element] = np.NaN
    col = colcom + coldif
    col = sorted(col)
    df = df.loc[:,col]
    df = df.drop_duplicates(subset=colcom)
    return df
    

In [1]:
dict_old = {}
dict_new = {}

df_old = rename_columns(df_old, dict_old)
df_new = rename_columns(df_new, dict_new)
df = pd.concat([df_old, df_new])

NameError: name 'rename_columns' is not defined

In [ ]:
columns=[]


def drop_dup_df(df):
    df = fit_df(df, columns)
    df = df.reset_index(drop=True)
    df = df.drop_duplicates()
    return df

df_old = drop_dup_df(df_old)
df_new = drop_dup_df(df_new)

df_new = df_new[df_new['ASSIGNED_TO'].isin(df_consultor['ASSIGNED_TO'])]

#df_old = df_new[df_old['ASSIGNED_TO'].isin(df_consultor['ASSIGNED_TO'])]

C:\Users\rafael.ferrari\AppData\Local\Temp\ipykernel_5192\369901452.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[each_element] = np.NaN
C:\Users\rafael.ferrari\AppData\Local\Temp\ipykernel_5192\369901452.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[each_element] = np.NaN
C:\Users\rafael.ferrari\AppData\Local\Temp\ipykernel_5192\369901452.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [ ]:
df_new['CLOSED'] = df_new['CLOSED'].replace('00-00-0000', np.NAN) #  0324-001003
df_new['CREATED'] = pd.to_datetime(df_new['CREATED'], format='%d-%m-%Y')
df_new['CLOSED'] = pd.to_datetime(df_new['CLOSED'], format='%d-%m-%Y')
df_tickets = pd.concat([df_new, df_old])


In [ ]:
# converter datas
df_tickets['CLOSED'].replace('00-00-0000', np.NAN, inplace=True)
df_tickets['CREATED_DATE_TIME'] = pd.to_datetime(df_tickets['CREATED'], format='mixed')  #, format='%d%m%Y %I:%M %p
df_tickets['CLOSED_DATE_TIME'] = pd.to_datetime(df_tickets['CLOSED'], format='mixed')

#df_tickets['INCIDENTDURATIONINDAYS'] = df_tickets['INCIDENTDURATIONINDAYS'].apply(lambda x: '{:.2}'.format(x).replace('.', ','))
#df_tickets['INCIDENTDURATIONINHOURS'] = df_tickets['INCIDENTDURATIONINHOURS'].apply(lambda x: '{:.2}'.format(x).replace('.', ','))
#df_tickets['TOTAL TASK TIME'] = df_tickets['TOTAL TASK TIME'].apply(lambda x: '{:.2}'.format(x).replace('.', ','))

In [ ]:
# apply the ASSIGNED TEAM rules
df_tickets = df_tickets[df_tickets['GROUP'].isin(['KMC GL SAP OPS SD', 'KMC GL SAP OPS LOGISTICS', 'KMC GL SAP OPS MM']) |
                        (df_tickets['DF_NAME']=='xpto')]
df_tickets['GROUP'] = ['KMC GL SAP OPS SD' if x == 'KMC GL SAP OPS LOGISTICS' else x for x in df_tickets['GROUP']]

In [ ]:
df_tickets.loc[df_tickets['CREATED_DATE_TIME']>df_tickets['CLOSED_DATE_TIME'], \
               ['DF_NAME', 'TICKET', 'CREATED', 'CLOSED', 'CREATED_DATE_TIME', 'CLOSED_DATE_TIME']] #.groupby('DF_NAME')['TICKET'].count()

,DF_NAME,TICKET,CREATED,CLOSED,CREATED_DATE_TIME,CLOSED_DATE_TIME


In [ ]:
# Define Resolved Status to Closed Status
df_tickets['DATE TODAY'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#df_tickets['DATE TIME CLOSED'] = [dt_resolved if pd.isna(dt_closed) else dt_closed
#                                  for dt_resolved, dt_closed in zip(df_tickets['DATE TIME RESOLVED'], df_tickets['DATE TIME CLOSED'])]
df_tickets['AGING'] = [(dt_closed - dt_opened).days if not pd.isna(dt_closed) else (datetime.now() - dt_opened).days
                        for dt_opened, dt_closed in zip(df_tickets['CREATED_DATE_TIME'], df_tickets['CLOSED_DATE_TIME'])]
df_tickets.loc[df_tickets['STATUS'] == 'Resolved', 'STATUS'] = 'Closed'
df_tickets.loc[pd.isna(df_tickets['STATUS']), 'STATUS'] = 'In Progress'

dict_status = {'Closed':'Closed', 'In Progress':'In Progress', 'Finalizado':'Closed', 'Approval':'In Progress', 
               'Aguardando Atendimento':'In Progress', 'Aguardando Terceiros':'In Progress', 'Aguardando Cliente':'In Progress',
               'Transferência':'In Progress', 'Fluxo de aprovações no Cherwell - USA':'In Progress', 'Cancelado':'Canceled', 
               'Andamento':'In Progress', 'Acompanhamento':'In Progress', 'Fluxo de aprovações no Cherwell - Business 2024':'In Progress',
               'Fluxo de aprovações no Cherwell - Business':'In Progress'}
df_tickets['STATUS_AGG'] = df_tickets['STATUS'].replace(dict_status)

In [ ]:
def change_to_int(df, list_col):
    for each_col in list_col:
        df.loc[df[each_col]=='', each_col] = 0
        df[each_col] = df[each_col].astype('int')
    return df
    
df_tickets = change_to_int(df_tickets, ['PRIORITY'])

In [ ]:
df_tickets['CHERWELL'] = df_tickets['CHERWELL'].astype('str')

def ajusta_inc_sr(df, col_name, col_new_name):
    list_inc_sr = []
    for each_val in df[col_name]:
        if each_val[0:1] == 'S':
            list_inc_sr.append('SR')
        elif each_val[0:1] == 'I':
            list_inc_sr.append('INC')
        elif each_val=='':
            list_inc_sr.append('INC')
        elif each_val[0:1] in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']:
            list_inc_sr.append('INC')
        else:
            list_inc_sr.append(np.NaN)
    df[col_new_name] = list_inc_sr
    return df

df_tickets = ajusta_inc_sr(df_tickets, 'CHERWELL', 'INC_SR')

In [ ]:
df_tickets.reset_index(inplace=True, drop=True)

In [ ]:
# Bulding a Day Calendar with the paremeters already defined before
from dateutil.relativedelta import relativedelta

anoini = date.today().year - 3
anofim = date.today().year + 1

dateiniday = datetime(anoini, 1, 1)
datefimday = datetime(anofim, 12, 31)

current_year = datetime.today().year
current_month = datetime.today().month
current_date = datetime.today()
l12m = current_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0) + relativedelta(months=-12)
l6m = l12m + relativedelta(months= 6)
lm_1_ini = current_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0) + relativedelta(months=0)
lm_1_fim = current_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0) + relativedelta(months=+1) + relativedelta(days=-1) 

def convert_str_datetime(data_str):
    formato = "%d/%m/%Y %H:%M:%S"
    data_formatada = datetime.strptime(data_str, formato)
    return data_formatada

def zero_sec(df, colunas):
    for col in pd.DataFrame(df[colunas]).columns:
        df[col] = pd.to_datetime(df[col]).dt.floor('min')
    return df

def calendar():
    date_range_minute = pd.date_range(start=dateiniday, end=datefimday, freq='T')

    df_calendar_minute = pd.DataFrame({'DATETIME': date_range_minute})

    df_calendar_minute['DATETIME'] = pd.to_datetime(df_calendar_minute['DATETIME']) #, format='%Y-%m-%d'
    df_calendar_minute['DATE'] = df_calendar_minute['DATETIME'].dt.date

    df_calendar_minute['FLAG_CURRENT_MONTH_DATE'] = False
    df_calendar_minute['FLAG_CURRENT_YEAR_DATE'] = False
    df_calendar_minute['FLAG_L12M_DATE'] = False
    df_calendar_minute['FLAG_LM1_DATE'] = False
    df_calendar_minute.loc[(df_calendar_minute['DATETIME'].dt.year == current_year) &
                    (df_calendar_minute['DATETIME'].dt.month  == current_month),
                    'FLAG_CURRENT_MONTH_DATE'] = True
    df_calendar_minute.loc[(df_calendar_minute['DATETIME'].dt.year == current_year),
                    'FLAG_CURRENT_YEAR_DATE'] = True
    df_calendar_minute.loc[(df_calendar_minute['DATETIME'].dt.date >= l12m.date()) & 
                           (df_calendar_minute['DATETIME'].dt.date <= current_date.date()),
                    'FLAG_L12M_DATE'] = True
    df_calendar_minute.loc[(df_calendar_minute['DATETIME'].dt.date >= lm_1_ini.date()) & 
                            (df_calendar_minute['DATETIME'].dt.date <= lm_1_fim.date()), 'FLAG_LM1_DATE'] = True
    df_calendar_minute['YEAR_MONTH'] = df_calendar_minute['DATETIME'].dt.to_period('M')
    df_calendar_minute['YEAR'] = df_calendar_minute['DATETIME'].dt.year
    df_calendar_minute['MONTH'] = df_calendar_minute['DATETIME'].dt.month
    df_calendar_minute['DAY'] = df_calendar_minute['DATETIME'].dt.day
    df_calendar_minute['MONTH_ABBR'] = df_calendar_minute['DATETIME'].dt.strftime('%b')
    df_calendar_minute['TIME'] = df_calendar_minute['DATETIME'].dt.time
    df_calendar_minute['WEEKDAY'] = df_calendar_minute['DATETIME'].dt.weekday
    df_calendar_minute['WEEK'] = (pd.to_datetime(df_calendar_minute['DATE']).dt.day//7 + 1).astype(int)
    df_calendar_minute = df_calendar_minute.sort_values(by='DATETIME', ascending = True)
    df_calendar_minute = df_calendar_minute.reset_index(drop=True)


    list_holiday = ['2023-01-25', '2023-02-21', '2023-04-07', '2023-04-21', 
                    '2023-05-01', '2023-06-08', '2023-09-07', '2023-10-12',
                    '2023-11-02', '2023-11-15', '2023-12-25', '2022-01-01',
                    '2022-03-01', '2022-04-15', '2022-04-21', '2022-06-16',
                    '2022-09-07', '2022-10-12', '2022-11-02', '2022-11-15',
                    '2022-12-25', '2024-01-01', '2024-03-29', '2024-04-21',
                    '2024-05-01', '2024-09-07', '2024-10-12', '2024-10-12',
                    '2024-11-02', '2024-11-15', '2024-12-25', '2024-07-09',
                    '2024-01-25', '2024-05-30', '2024-11-20', '2024-02-13']

    formato = '%Y-%m-%d'
    list_holiday = [datetime.strptime(data, formato).date() for data in list_holiday]

    # BUSINESS_HOUR
    df_calendar_minute['BUSINESS_MIN'] = [1 if (df_calendar_minute['WEEKDAY'].iloc[i] not in (5, 6) and
                                    df_calendar_minute['DATE'].iloc[i] not in list_holiday and
                                    ((df_calendar_minute['TIME'].iloc[i].hour >= 8 and df_calendar_minute['TIME'].iloc[i].hour < 12) or
                                    (df_calendar_minute['TIME'].iloc[i].hour >= 14 and df_calendar_minute['TIME'].iloc[i].hour < 18)))
                                    else 0
                                    for i in range(len(df_calendar_minute['DATE']))]

    last_index = 0
    new_index = []

    for flag in df_calendar_minute['BUSINESS_MIN']:
        if flag == 1:
            last_index += 1
            new_index.append(last_index)
        else:
            new_index.append(last_index)

    df_calendar_minute = zero_sec(df_calendar_minute, 'DATETIME')
            
    df_calendar_minute.insert(0, 'INDEX', df_calendar_minute.index)
    df_calendar_minute['FLAG_L6M_DATE'] = False
    df_calendar_minute.loc[df_calendar_minute['DATE'] >= l6m.date(),'FLAG_L6M_DATE'] = True 

    df_calendar_day = df_calendar_minute.copy()
    df_calendar_day.drop(['DATETIME', 'INDEX', 'TIME', 'BUSINESS_MIN'], axis=1, inplace=True)
    df_calendar_day.drop_duplicates(inplace=True)
    df_calendar_day.reset_index(inplace=True, drop=True)
    df_calendar_day.insert(0, 'INDEX', df_calendar_day.index)

    df_calendar_month = pd.DataFrame(df_calendar_day['MONTH_ABBR'].copy())
    df_calendar_month.drop_duplicates(inplace=True)
    df_calendar_month.reset_index(inplace=True, drop=True)
    df_calendar_month.insert(0, 'INDEX', df_calendar_month.index + 1)

    df_calendar_year_month = pd.DataFrame(df_calendar_day[['YEAR', 'YEAR_MONTH', 'MONTH', 'MONTH_ABBR', 
                                                        'FLAG_L12M_DATE', 'FLAG_LM1_DATE']].copy())
    #df_calendar_year_month = df_calendar_year_month[df_calendar_year_month['FLAG_L12M_DATE']==True]                                              
    df_calendar_year_month.drop_duplicates(inplace=True)
    df_calendar_year_month.reset_index(inplace=True, drop=True)
    df_calendar_year_month.insert(0, 'INDEX', df_calendar_year_month.index + 1)

    df_calendar_minute.to_csv(db_pbi + '\calendar_minute.csv', sep=';', index=False)
    df_calendar_day.to_csv(db_pbi + '\calendar_day.csv', sep=';', index=False)
    df_calendar_month.to_csv(db_pbi + '\calendar_month.csv', sep=';', index=False)
    df_calendar_year_month.to_csv(db_pbi + '\calendar_year_month.csv', sep=';', index=False)

    return [df_calendar_minute, df_calendar_day, df_calendar_month, df_calendar_year_month]

In [ ]:
def change_str_datetime(list_df):
    for each_df in list_df:
        for each_col in each_df.columns:
            if each_col == 'DATETIME':
                each_df['DATETIME'] = pd.to_datetime(each_df['DATETIME'])
            elif each_col == 'DATE':
                each_df['DATE'] = pd.to_datetime(each_df['DATE']).dt.date
    return list_df

def to_check_calendar():
    dict_calendar =  {'df_calendar_minute':'calendar_minute.csv', 'df_calendar_day':'calendar_day.csv', 
                      'df_calendar_month':'calendar_month.csv', 'df_calendar_year_month':'calendar_year_month.csv'}
    tof = True
    list_return_calendar = []
    for key, value in dict_calendar.items():
        c = os.path.join(db_pbi, value)
        tof = tof * os.path.exists(c)
        if tof == True:
            key = pd.read_csv(c, sep=';', encoding='latin-1')
            list_return_calendar.append(key)
        else:
            return calendar()

    for each_calendar in list_return_calendar:
        if 'DATE' in each_calendar.columns:
            formato = '%Y-%m-%d'
            max_date = datetime.strptime(each_calendar['DATE'].max(), formato).date()
            if max_date != datefimday.date():
                return calendar()
            max_date = 0
            formato = ''
        elif 'YEAR_MONTH' in each_calendar.columns:
            formato = '%Y-%m'
            max_date = each_calendar['YEAR_MONTH'].max()
            if max_date != current_date.strftime(formato):
                return calendar()
            max_date = 0
            formato = ''
        elif 'INDEX' in each_calendar.columns:
            #list_month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            val_val = set(range(1,13))
            index = set(each_calendar['INDEX'])
            if val_val != index:
                return calendar()
    list_return_calendar = change_str_datetime(list_return_calendar)
    return calendar()

[df_calendar_minute, df_calendar_day, df_calendar_month, df_calendar_year_month] = to_check_calendar()

In [ ]:
# Business Hour Index
def bsns_hour_index(df):
    last_index = 0
    new_index = []
    for flag in df_calendar_minute['BUSINESS_MIN']:
        if flag == 1:
            last_index += 1
            new_index.append(last_index)
        else:
            new_index.append(last_index)
    return new_index

df_calendar_minute['INDEX_BSNS_HOUR'] = bsns_hour_index(df_calendar_minute)

In [ ]:
df_tickets['CREATED'] = pd.to_datetime(df_tickets['CREATED'], format='mixed')
df_tickets['CLOSED'] = pd.to_datetime(df_tickets['CLOSED'], format='mixed')

In [ ]:

# Create OPENED Table
df_opened = df_tickets[(~df_tickets['CREATED'].isna())].copy()
df_opened['REFERENCE_AT_DATETIME'] = df_opened['CREATED'] #df_opened['REFERENCE_AT_DATE'] = pd.to_datetime(df_opened['REFERENCE_AT_DATE'])
df_opened['STATUS_MONTH'] = 'OPENED'
df_opened['COUNT_TICKETS'] = 1
df_opened['YTD'] = ['YTD' if datetime.now().year == pd.to_datetime(data).year
                    else (pd.to_datetime(data).year - datetime.now().year) 
                    for data in df_opened['REFERENCE_AT_DATETIME']]

# Create Closed Table
df_closed = df_tickets[(~df_tickets['CLOSED'].isna())].copy()
df_closed['REFERENCE_AT_DATETIME'] = df_closed['CLOSED'] #df_closed['REFERENCE_AT_DATE'] = pd.to_datetime(df_closed['REFERENCE_AT_DATE'])
df_closed['STATUS_MONTH'] = 'CLOSED'
df_closed['COUNT_TICKETS'] = 1
df_closed['YTD'] = ['YTD' if datetime.now().year == pd.to_datetime(data).year
                    else (pd.to_datetime(data).year - datetime.now().year) 
                    for data in df_closed['REFERENCE_AT_DATETIME']]

# Create BACKLOG Table
df_backlog = df_tickets[((df_tickets['CLOSED'] >= (pd.to_datetime(df_tickets['CREATED_DATE_TIME']) + pd.offsets.MonthBegin(1)).dt.to_period('M').dt.to_timestamp()) &
                        ~df_tickets['CLOSED'].isna()) |
                        (df_tickets['CLOSED'].isna())].copy()

df_backlog['REFERENCE_AT_DATETIME'] = df_backlog['CREATED'] #df_closed['REFERENCE_AT_DATE'] = pd.to_datetime(df_closed['REFERENCE_AT_DATE'])
df_backlog['STATUS_MONTH'] = 'BACKLOG'
df_backlog['COUNT_TICKETS'] = 1
df_backlog['YTD'] = ['YTD' if datetime.now().year == pd.to_datetime(data).year
                    else (pd.to_datetime(data).year - datetime.now().year) 
                    for data in df_backlog['REFERENCE_AT_DATETIME']]
# LOOK FOR BACKLOG LINES WITH MORE THAN 1 MONTH OF DIFFERENCE BETWEEN RESOLVED/CLOSED AND OPENED TO INPUT A NEW LINE IN THE GAP MONTH 
def dup_backlog_line(df):
    x = 0
    new_series = []
    for idx, row in df.iterrows():
        x = int(row['datediff'] - 1)
        for i in range(x):
            row['REFERENCE_AT_DATETIME'] = (pd.to_datetime(row['CREATED']) + pd.DateOffset(months=i+1)).to_period('M').to_timestamp()
            #print(str(row['NUMBER']) + ' - ' + str(row['OPENED_AT']) + ' - ' + str(row['REFERENCE_AT_DATETIME']) + ' - ' +  str(row['datediff']) + ' - ' + str(i) + ' - '+ str(x))
            new_series.append(row.copy())
    return new_series

df_backlog_date_dif = df_backlog.copy()
df_backlog_date_dif['dateClosedOpened'] = df_backlog_date_dif['CLOSED']
df_backlog_date_dif.loc[df_backlog_date_dif['CLOSED'].isna(),'dateClosedOpened'] = datetime.now()
df_backlog_date_dif.loc[df_backlog_date_dif['dateClosedOpened'].isna(),'dateClosedOpened'] = df_backlog_date_dif['CLOSED']
df_backlog_date_dif['datediff'] = ((pd.to_datetime(df_backlog_date_dif['dateClosedOpened'], format='mixed').dt.year - 
                                pd.to_datetime(df_backlog_date_dif['CREATED'], format='mixed').dt.year)*12 + 
                                pd.to_datetime(df_backlog_date_dif['dateClosedOpened'], format='mixed').dt.month - 
                                pd.to_datetime(df_backlog_date_dif['CREATED'], format='mixed').dt.month)
df_backlog_date_dif = df_backlog_date_dif[df_backlog_date_dif['datediff']>1].copy()


df_auxiliar = pd.DataFrame(dup_backlog_line(df_backlog_date_dif))
df_auxiliar.drop(['dateClosedOpened','datediff'], axis=1, inplace=True)
df_backlog  = pd.concat([df_backlog, df_auxiliar], axis=0)

# UNION
df = pd.concat([df_opened, df_closed, df_backlog], ignore_index=True)

# REFERENCE_AT_DATETIME REFERENCE_AT_DATE
df['REFERENCE_AT_DATETIME'] = pd.to_datetime(df['REFERENCE_AT_DATETIME'])
df['REFERENCE_AT_DATE'] = df['REFERENCE_AT_DATETIME'].dt.date
df['REF_Y_M'] = df['REFERENCE_AT_DATETIME'].dt.strftime('%Y-%m')
#print(df_gerot_sctask.loc[df_gerot_sctask['NUMBER']=="SCTASK1535820", ['NUMBER', 'STATUS_MONTH', 'OPENED_AT', 'CLOSED_AT', 'REFERENCE_AT_DATETIME']])


In [ ]:
df['CREATED'] = pd.to_datetime(df['CREATED'], format='mixed').dt.date
df['CLOSED'] = pd.to_datetime(df['CLOSED'], format='mixed').dt.date
df['RESOLVED'] = pd.to_datetime(df['RESOLVED'], format='mixed')

In [ ]:
def class_aging(df, coluna_aging, coluna_nome):
    classificacao = pd.cut(df[coluna_aging], bins=[-1, 15, 30, 45, 60, df[coluna_aging].max()+1], labels=['<15', '<30', '<45', '<60', '>60'])
    df[coluna_nome] = classificacao
    return df

df = class_aging(df, 'AGING', 'CLASS_AGING')

df = df.sort_values(by='REFERENCE_AT_DATETIME', ascending=True)

df['FLAG_OPENED'] = False
df.loc[df['STATUS_AGG'].isin(['In Progress', 'Pending']) &
       df['STATUS_MONTH'].isin(['OPENED']), 'FLAG_OPENED'] = True

mask = df.loc[(df['STATUS_AGG']=='OPENED') &
       (df['STATUS_MONTH']=='BACKLOG'), :].groupby(['TICKET'])['REFERENCE_AT_DATETIME'].idxmax()
df.iloc[mask, :].loc[:,'FLAG_OPENED'] = True

In [ ]:
df['DESCRIPTION'] = df['DESCRIPTION'].str.replace('\n', ' ')
df['DESCRIPTION'] = df['DESCRIPTION'].str.replace(';', ',')
df['DESCRIPTION'] = df['DESCRIPTION'].str.replace('\r', ' ')
#df['DESCRIPTION'] = "'" + df['DESCRIPTION'] + "'"

In [ ]:
df = pd.merge(df, df_consultor, left_on='ASSIGNED_TO', right_on='ASSIGNED_TO', how='left')

In [ ]:
df.to_csv(rf'{filepath}', sep=';', index=False)
#df[['STATUS_MONTH', 'TICKET', 'DF_NAME','REF_Y_M','COUNT_TICKETS', 'GROUP']].to_csv(rf'{filepathteste}', sep=';', index=False)

In [ ]:
df.to_csv(hist_path + arch_name + str(dt_arch) + '.csv', index=False)

In [ ]:
import sys

# Construct the full path to the script (replace with your actual path)
script_path = rf""

# Prepend the script's directory to the system path (if necessary)
if script_path not in sys.path:
    sys.path.append(os.path.dirname(script_path))

# Now you can import the functions from Funcoes.py
from Funcoes import send_email

#send_email(list_email, email_customer)